# Best Tutorial: [Official Website](https://www.djangoproject.com/start/)


# Create project

```shell
# installation: 
> pip install django

# versioning
> python -m django --version

# Creating a project
> django-admin startproject ModelDeployment

# start testing environment
> python manage.py runserver
```


# Build venv

```shell
# change to the ModelDeployment
> cd ModelDeployment

# if not install virtualenv
> pip insatll virutalenv

# build venv
> virtualenv venv

# activate venv
> venv\Scripts\activate

# install django
> pip install django numpy lightgbm matplotlib
```

# Creat application 

```shell
# creat application
> python manage.py startapp minst
```



# Open your Editor (VScode)
```shell
# open vscode
> code .
```


# Observer the directory structure
```
ModelDeployment/
    manage.py                    # execute any django script(runserver, shell, or makemigrations, migrate)
    ModelDeployment/
        __init__.py
        settings.py              # setting of your website (DB, APP, HOST, BASE_DIR, STATIC. MEDIA)
        urls.py                  # first level of router
        wsgi.py                  # locate the whole project when deployment of the website
    minst/
        __init__.py
        admin.py
        apps.py
        models.py                # ORM to the DB
        tests.py                 # testind scripts
        urls.py                  # second level router
        views.py                 # main script executing
        migrations/              # DB ORM modification history
            __init__.py
```

# [Write your first view](https://docs.djangoproject.com/en/2.2/intro/tutorial01/)

1. minst/views.py 

```python
from django.shortcuts import render
from django.http import HttpResponse
def index(request):
    return HttpResponse("Hello")
```

2. minst/urls.py (creating)

```python
from django.urls import path
from . import views
urlpatterns = [
    path('', views.index, name='index'),
]
```

3. ModelDeployment/urls.py

```python
from django.contrib import admin
from django.urls import include, path
urlpatterns = [
    path('minst/', include('minst.urls')),
    path('admin/', admin.site.urls),
]
```

4. testing

```python
# runserver
> python manage.py runserver

# go to website: 
http://127.0.0.1:8000/minst/
```


# Serve the image & model
1. ModelDeployment/settings.py bottom

```
STATIC_ROOT = os.path.join(BASE_DIR, "static/")
STATIC_URL = '/static/'
MEDIA_ROOT = os.path.join(BASE_DIR,'media')
MEDIA_URL = '/media/'
```


# [Templates](https://docs.djangoproject.com/en/2.2/intro/tutorial03/)
1. minst/templates/minst/predict.html: https://docs.djangoproject.com/en/2.2/topics/forms/

```python
<form action="" method="post">
    <input type='file'/>
    <button type='submit'>Submit</button>
</form>
```

2. minst/views.py

```python
from django.shortcuts import render
from django.http import HttpResponse

def index(request):
    return HttpResponse("Hello")

def predict(request):
    template = loader.get_template('minst/predict.html')
    context = {}
    return HttpResponse(template.render(context, request))
```

3. minst/urls.py

```python
from django.urls import path
from . import views
urlpatterns = [
    path('', views.index, name='index'),
    path('predict', views.predict, name='predict'),
]
```

4. ModelDeployment/settings.py

```python
INSTALLED_APPS = [
    'django.contrib.admin',
    'django.contrib.auth',
    'django.contrib.contenttypes',
    'django.contrib.sessions',
    'django.contrib.messages',
    'django.contrib.staticfiles',
    'minst.apps.MinstConfig',
]
```



# Deal with [POST FILES](https://docs.djangoproject.com/en/2.2/topics/http/file-uploads/)
1. add POST in predict function: https://docs.djangoproject.com/en/2.2/topics/http/file-uploads/

```python
from django.template import loader
from django.http import HttpResponse
from django.conf import settings
import os

def index(request):
    return HttpResponse("Hello")

def predict(request):
    if request.method == "GET":
        template = loader.get_template('minst/predict.html')
        context = {}
        return HttpResponse(template.render(context, request))
    elif request.method == "POST":
        f = request.FILES['user_img']
        with open(os.path.join(settings.MEDIA_ROOT, 'test.png'), 'wb+') as destination:
            for chunk in f.chunks():
                destination.write(chunk)
        return HttpResponse("Succuess!")
```

2. create media directory

3. deal with csrf issue

```python
{% csrf_token %}

```

4. deal with file upload request.FILES empty: https://stackoverflow.com/questions/2010747/empty-request-files-with-django-upload-forms

```html
<form action="" method="post" enctype="multipart/form-data">
{% csrf_token %}
    <input type='file' name='user_img'/>
    <button type='submit'>Submit</button>
</form>

```

# Use model to predict
1. put model in media/models/lgb_model.txt
2. import lightgbm in views.py

```python
import lightgbm
import numpy as np
from matplotlib import pyplot as plt
```

3. load lgb_models.txt

```python
lgb_model = lightgbm.Booster(model_file='lgb_model.txt')
```

4. open shell to try it

```shell
> python manage.py shell
```

5. add predict script

```python
f = request.FILES['user_img']
# img_fp = os.path.join(settings.MEDIA_ROOT, 'test.png')
# with open(img_fp, 'wb+') as destination:
#     for chunk in f.chunks():
#         destination.write(chunk)
# img = plt.imread(img_fp)[:, :, 0]

img = plt.imread(f)[:, :, 0]
lgb_model = lightgbm.Booster(model_file=os.path.join(settings.MEDIA_ROOT, 'models', 'lgb_model.txt'))
y_pred = lgb_model.predict([img.flatten()])
y_pred = np.argmax(y_pred, axis=1)[0]
return HttpResponse(str(y_pred))

```
